In [1]:
import torch
from ultralytics import YOLO
import numpy as np

device: str = "mps" if torch.backends.mps.is_available() else "cpu"

model = YOLO('./models/yolov8x-seg.pt')  # load a pretrained YOLOv8n segmentation model

In [2]:
import cv2
img = cv2.imread('../images/specialcontent/istockphoto-181610647-612x612.jpg')
H, W, _ = img.shape
results = model(img)


0: 640x480 1 vase, 637.9ms
Speed: 3.0ms preprocess, 637.9ms inference, 9.0ms postprocess per image at shape (1, 3, 640, 480)


In [3]:
for result in results:
    for j, mask in enumerate(result.masks.data):
        mask = mask.cpu().numpy() * 255
        mask  =cv2.resize(mask, (W, H))
        cv2.imwrite('./output.png', mask)


In [4]:
for result in results:
    for j, mask in enumerate(result.masks.data):
        # Resize the mask to match the original image dimensions
        mask = cv2.resize(mask.cpu().numpy(), (W, H))

        # Create a red-colored mask
        red_mask = np.zeros_like(img)
        red_mask[:, :, 2] = 255  # Set the red channel to 255

        # Apply the mask to make the object red
        red_object = cv2.bitwise_and(red_mask, red_mask, mask=(mask > 0).astype(np.uint8))

        cv2.imshow("object",red_object)

        cv2.waitKey(0) 
        
        cv2.destroyAllWindows()

        # Invert the mask to create a black background
        black_background = cv2.bitwise_not((mask > 0).astype(np.uint8))

        # Convert black_background to the same data type as img
        black_background = black_background.astype(np.uint8)

        # Extract the region from the original image
        original_region = cv2.bitwise_and(img, img, mask=black_background)

        # Add the red-colored object to the original region
        result_image = cv2.add(original_region, red_object)

        # Save or display the result image
        cv2.imwrite('./output.png', result_image)

In [1]:
import cv2
def imread(path):
    img = cv2.imread(path).astype(np.float)
    if len(img.shape) == 2:
        # grayscale
        img = np.dstack((img,img,img))
    elif img.shape[2] == 4:
        # PNG with alpha channel
        img = img[:,:,:3]
    return img

def imsave(path, img):
    img = np.clip(img, 0, 255).astype(np.uint8)
    Image.fromarray(img).save(path, quality=95)